# Corporate Credit Early Warning System (EWS)

## Project Overview

Hệ thống cảnh báo sớm rủi ro tín dụng doanh nghiệp (Corporate Credit EWS) được xây dựng theo chuẩn Basel, sử dụng Machine Learning để dự đoán xác suất vỡ nợ (PD - Probability of Default) trong vòng 12 tháng tới.

**Mục tiêu chính:**
- Dự đoán khả năng vỡ nợ của khách hàng doanh nghiệp (event horizon: 12 tháng)
- Phân loại rủi ro thành 3 tiers: **Green** (an toàn), **Amber** (cảnh báo), **Red** (nguy hiểm)
- Đưa ra khuyến nghị hành động cụ thể cho Risk Management team

**Tech Stack:**
- Python 3.13
- LightGBM (classification model)
- SHAP (model explainability)
- Sklearn (calibration, metrics)
- Pandas/Numpy (data processing)

## Pipeline Architecture

Dự án được tổ chức theo **end-to-end ML pipeline** với 7 bước chính:

```
1. Data Generation (generate_data.py)
   ↓
2. Feature Engineering (feature_engineering.py)
   ↓
3. Model Training + Calibration (train_baseline.py)
   ↓
4. Model Explainability (explain.py)
   ↓
5. [Optional] Make Raw Scores (make_scores_raw.py)
   ↓
6. [Optional] Re-calibration (calibrate.py)
   ↓
7. Production Scoring (scoring.py)
```

### Data Flow
- **Raw Data** → `data/raw/` (fin_quarterly, credit_daily, cashflow_daily, covenant, labels)
- **Features** → `data/processed/` (feature_ews.parquet)
- **Models** → `artifacts/models/` (model_lgbm.pkl, SHAP artifacts)
- **Scores** → `artifacts/scoring/` (ews_scored_YYYY-MM-DD.csv)

## Step 1: Data Generation

Module: `src/generate_data.py`

### Overview

Tạo dữ liệu synthetic (giả lập) để train và test model EWS. Dữ liệu được thiết kế sát với thực tế nghiệp vụ tín dụng doanh nghiệp và tuân thủ các nguyên tắc Basel.

### Why Synthetic Data?

#### Lợi ích và hạn chế
Lợi ích của việc sử dụng dữ liệu tổng hợp là tránh được vấn đề bảo mật và rủi ro tuân thủ do sử dụng các dữ liệu nội bộ từ ngân hàng. Bên cạnh đó, ta có thể biết chính xác ground truth dữ liệu đầu vào của mô hình (ở đây là event_12m), điều mà vô cùng quan trọng khi xây dựng các thuật toán học máy như LightGBM. Ngoài ra, chúng ta giải quyết được vấn đề scaling bằng việc dễ dàng tạo 1K, 10K, 100K dữ liệu khách hàng, điều mà khá khó khăn với những dự án có kinh phí thấp và khả năng tiếp cận hạn chế đối với dữ liệu thật. Một lợi thế khác khi sử dụng dữ liệu tổng hợp là khả năng thực hiện các edge cases và stress scenarios: những trường hợp dữ liệu nằm ở biên của phân phối thông thường, rất ít khi xuất hiện trong dữ liệu thực tế.

Hạn chế lớn nhất của việc sử dụng dữ liệu tổng hợp là sự khác biệt về phân phối (distribution shift) so với dữ liệu thực tế. Nếu dữ liệu tổng hợp không mô phỏng chính xác mối quan hệ phức tạp và các sắc thái ẩn trong dữ liệu gốc của ngân hàng, mô hình được huấn luyện trên đó có thể hoạt động kém hiệu quả hoặc đưa ra các dự đoán sai lệch nghiêm trọng khi áp dụng vào môi trường thực tế. 

#### Vai trò
Dữ liệu tổng hợp có vai trò thiết yếu trong nhiều giai đoạn của dự án khoa học dữ liệu. Nó được sử dụng để chứng minh tính khả thi (Proof-of-concept) của mô hình hoặc giải pháp trước khi áp dụng lên dữ liệu thật, giúp giảm thiểu rủi ro. Ngoài ra, dữ liệu này là công cụ lý tưởng để huấn luyện các đội ngũ mới mà không cần tiếp xúc với thông tin nhạy cảm.

---

### Customer Profile Configuration

```python
@dataclass
class Config:
    n_customers: int = 1000
    sectors: Tuple[str, ...] = (
        "MFG",  # Manufacturing (18%)
        "TRA",  # Trading (12%)
        "CON",  # Construction (10%)
        "AGR",  # Agriculture (8%)
        "ENG",  # Engineering (8%)
        "CHE",  # Chemicals (10%)
        "RET",  # Retail (12%)
        "LOG",  # Logistics (10%)
        "TEL",  # Telecom (6%)
        "IT"    # Information Technology (6%)
    )
    size_buckets: Tuple[str, ...] = ("SME", "Corp")
    size_probs: Tuple[float, ...] = (0.8, 0.2)  # 80% SME, 20% Corp
```

**Sector Risk Premiums:**
Mỗi sector có risk premium khác nhau (thêm vào debt_mult):
- **Low risk:** ENG (0.0), MFG (0.0), TEL (-0.01), IT (-0.02)
- **Medium risk:** CHE (0.02), LOG (0.02), RET (0.03), CON (0.03)
- **Higher risk:** AGR (0.04), TRA (0.05)

**Size Characteristics:**
- **SME (Small-Medium Enterprise):**
  - Base revenue: lognormal(mean=10.5, σ=0.5)
  - Debt multiplier: 0.8 + 0.4 = 1.2
  - Higher default risk due to less stability
  
- **Corp (Large Corporate):**
  - Base revenue: lognormal(mean=12, σ=0.5)  → ~2.7x larger
  - Debt multiplier: 0.8 + 0.6 = 1.4
  - More stable but higher leverage

---

### Data Tables Generated

#### 1. **fin_quarterly.parquet** - Báo cáo tài chính theo quý

**Time Range:** 12 quarters (3 years) history ending at `2025-06-30`

**Columns (15 total):**

| Column | Description | Generation Logic |
|--------|-------------|------------------|
| `customer_id` | Unique ID (C0001-C1000) | Sequential |
| `fq_date` | Quarter end date | 2022-09-30 → 2025-06-30 |
| `sector_code` | Industry sector | Random từ 10 sectors |
| `size_bucket` | SME or Corp | 80/20 split |
| `revenue` | Quarterly revenue | Growth ~2% QoQ + noise |
| `cogs` | Cost of Goods Sold | ~75% of revenue |
| `ebitda` | Earnings Before Interest, Tax, D&A | ~15% margin + noise |
| `ebit` | Earnings Before Interest & Tax | EBITDA - D&A (proxy 2% revenue) |
| `interest_expense` | Quarterly interest | Debt × 8% annual / 4 |
| `total_debt` | Total outstanding debt | Revenue × (0.3 + sector_risk) × debt_mult |
| `current_assets` | AR + Inventory + Cash | Function of revenue/COGS |
| `current_liab` | AP + short-term liabilities | Function of COGS/revenue |
| `inventory` | Inventory level | ~12% of COGS |
| `ar` | Accounts Receivable | ~18% of revenue (DSO ~66 days) |
| `ap` | Accounts Payable | ~20% of COGS (DPO ~73 days) |

**Realism Features:**
- ✅ **Growth patterns:** QoQ growth ~2% ± 5% noise
- ✅ **Profitability:** EBITDA margin 15% ± 7%
- ✅ **Leverage:** Debt/Revenue varies by sector
- ✅ **Working capital:** Realistic DSO, DPO, Inventory turnover

---

#### 2. **credit_daily.parquet** - Hành vi tín dụng hàng ngày

**Time Range:** 
- Observation window: 180 days before `asof_date` (2025-01-02 → 2025-06-30)
- Label window: 365 days after `asof_date` (2025-07-01 → 2026-06-30)
- **Total:** 545 days per customer

**Columns (7 total):**

| Column | Description | Generation Logic |
|--------|-------------|------------------|
| `customer_id` | Customer ID | - |
| `date` | Business date | Daily from start to end |
| `limit` | Credit limit | 30% of latest revenue × randomness |
| `utilized` | Amount used | Limit × utilization_rate |
| `breach_flag` | 1 if utilized > limit | Binary indicator |
| `dpd_days` | Days Past Due | Markov chain: 98.5% stay/improve, 1.5% deteriorate |
| `product_type` | OD/TERM/TR_LOAN | 70% Overdraft, 20% Term Loan, 10% Trade Finance |

**DPD Markov Process:**
```python
# Each day:
if random() < 0.985:
    dpd = max(0, dpd - binomial(1, p=0.3))  # 30% chance giảm 1 ngày
else:
    dpd += choice([1, 3, 7], p=[0.6, 0.3, 0.1])  # 60% +1, 30% +3, 10% +7
```

**Utilization Pattern:**
```python
util_level = beta(2, 2)  # Centered around 0.5
seasonal = sin(day_of_year/365 * 2π) * 0.05  # ±5% seasonality
daily_util = util_level + seasonal + noise(0, 0.05)
```

---

#### 3. **cashflow_daily.parquet** - Dòng tiền hàng ngày

**Time Range:** Same as credit_daily (545 days)

**Columns (4 total):**

| Column | Description | Generation Logic |
|--------|-------------|------------------|
| `customer_id` | Customer ID | - |
| `date` | Business date | Daily |
| `inflow` | Cash inflow | Daily avg revenue × seasonality × noise |
| `outflow` | Cash outflow | ~90% of inflow × noise |

**Seasonality Model:**
```python
daily_mean = annual_revenue / 365 × uniform(0.6, 1.1)
seasonal_factor = 1 + 0.2 × sin(day_of_year/365 × 2π)
inflow = max(0, normal(daily_mean × seasonal_factor, σ=daily_mean×0.3))
outflow = max(0, normal(inflow × 0.9, σ=daily_mean×0.25))
```

**Use cases:**
- Detect sudden revenue drops (inflow_drop_60d)
- Monitor burn rate (outflow > inflow)
- Identify cashflow volatility

---

#### 4. **covenant.parquet** - Covenant tracking

**Time Range:** Daily for observation + label windows

**Columns (7 total):**

| Column | Description | Threshold | Breach Logic |
|--------|-------------|-----------|--------------|
| `customer_id` | Customer ID | - | - |
| `date` | Business date | - | - |
| `icr` | Interest Coverage | ≥ 1.5 | 1 if < 1.5 |
| `dscr` | Debt Service Coverage | ≥ 1.2 | 1 if < 1.2 |
| `leverage` | Debt/EBITDA | ≤ 4.0 | 1 if > 4.0 |
| `breach_icr` | ICR breach flag | - | Binary |
| `breach_dscr` | DSCR breach flag | - | Binary |
| `breach_leverage` | Leverage breach flag | - | Binary |

**Why Important:**
- Covenant breach = Early warning signal
- Typical in loan agreements
- Triggers renegotiation or penalties

---

#### 5. **labels.parquet** - Target variable

**Columns (3 total):**

| Column | Description | Logic |
|--------|-------------|-------|
| `customer_id` | Customer ID | - |
| `asof_date` | Snapshot date | 2025-06-30 |
| `event_h12m` | Default in 12M | 1 if DPD ≥ 90 for ≥ 30 consecutive days |

**Label Definition (Basel-compliant):**
```python
def compute_label(future_dpd_series):
    """
    future_dpd_series: DPD values for 365 days after asof_date
    """
    max_consecutive_90plus = 0
    current_streak = 0
    
    for dpd in future_dpd_series:
        if dpd >= 90:
            current_streak += 1
            max_consecutive_90plus = max(max_consecutive_90plus, current_streak)
        else:
            current_streak = 0
    
    return 1 if max_consecutive_90plus >= 30 else 0
```

**Additional Bumps (increase default probability):**
- **High utilization bump:** If util_rate > 90% at asof_date → +20% PD
- **Covenant breach bump:** If any covenant breached → +20% PD

**Expected Default Rate:** ~5-10% (typical for corporate portfolio)

---

### Output Files

All tables saved in both **Parquet** (preferred) and **CSV** (fallback):

```
data/raw/
├── fin_quarterly.parquet       # ~1000 rows × 12 quarters = 12K rows
├── credit_daily.parquet        # ~1000 customers × 545 days = 545K rows
├── cashflow_daily.parquet      # ~1000 customers × 545 days = 545K rows
├── covenant.parquet            # ~1000 customers × 545 days = 545K rows
└── labels.parquet              # 1000 rows (one per customer)
```

**File sizes:**
- Parquet: ~5-10 MB total (compressed)
- CSV: ~30-50 MB total (uncompressed)

---

### Usage Example

```bash
# Generate with default config (1000 customers)
python src/generate_data.py --output-dir data/raw

# Generate 5000 customers for stress test
python src/generate_data.py --n-customers 5000 --output-dir data/raw_large

# Custom end date
python src/generate_data.py --end-quarter 2024-12-31 --output-dir data/raw_2024
```

---

### Quality Checks

**After generation, verify:**

✅ **Completeness:** All 5 files generated  
✅ **Row counts:** Consistent customer_ids across tables  
✅ **Date ranges:** Correct observation (180d) + label (365d) windows  
✅ **Label distribution:** Default rate 5-10%  
✅ **Financial sanity:** Revenue > 0, EBITDA margin reasonable, Debt > 0  
✅ **DPD distribution:** Majority < 30, some 30-90, few > 90  

```python
# Quick checks
import pandas as pd

labels = pd.read_parquet('data/raw/labels.parquet')
print(f"Default rate: {labels['event_h12m'].mean():.1%}")  # Should be ~5-10%

credit = pd.read_parquet('data/raw/credit_daily.parquet')
print(f"Max DPD: {credit['dpd_days'].max()}")  # Should see some 90+ days
print(f"Breach rate: {credit['breach_flag'].mean():.1%}")  # Should be low ~1-5%
```

In [ ]:
# Example: Generate synthetic data và verify kết quả

import sys
import os
sys.path.append('../src')

print("=" * 80)
print("STEP 1: GENERATE SYNTHETIC DATA")
print("=" * 80)

# Command để generate data
print("\n📝 Command to generate data:")
print("python src/generate_data.py --n-customers 1000 --output-dir data/raw")

print("\n📊 Expected outputs:")
outputs = {
    "fin_quarterly.parquet": "~12,000 rows (1000 customers × 12 quarters)",
    "credit_daily.parquet": "~545,000 rows (1000 customers × 545 days)",
    "cashflow_daily.parquet": "~545,000 rows (1000 customers × 545 days)",
    "covenant.parquet": "~545,000 rows (1000 customers × 545 days)",
    "labels.parquet": "1,000 rows (1 row per customer)"
}

for file, desc in outputs.items():
    print(f"  ✓ data/raw/{file:30s} → {desc}")

print("\n" + "=" * 80)
print("VERIFICATION AFTER GENERATION")
print("=" * 80)

# Nếu data đã tồn tại, verify nó
data_dir = "../data/raw"
if os.path.exists(data_dir):
    try:
        import pandas as pd
        
        print("\n✅ Data directory found! Checking files...")
        
        # Check labels
        labels_path = f"{data_dir}/labels.parquet"
        if os.path.exists(labels_path):
            labels = pd.read_parquet(labels_path)
            default_rate = labels['event_h12m'].mean()
            print(f"\n📌 labels.parquet:")
            print(f"   - Total customers: {len(labels):,}")
            print(f"   - Default rate (event_h12m=1): {default_rate:.1%}")
            print(f"   - Expected: 5-10% ✓" if 0.05 <= default_rate <= 0.15 else "   - Warning: Outside expected range")
        
        # Check credit_daily
        credit_path = f"{data_dir}/credit_daily.parquet"
        if os.path.exists(credit_path):
            credit = pd.read_parquet(credit_path)
            print(f"\n📌 credit_daily.parquet:")
            print(f"   - Total rows: {len(credit):,}")
            print(f"   - Date range: {credit['date'].min()} to {credit['date'].max()}")
            print(f"   - Max DPD: {credit['dpd_days'].max()} days")
            print(f"   - Breach rate: {credit['breach_flag'].mean():.1%}")
            print(f"   - Avg utilization: {(credit['utilized']/credit['limit']).mean():.1%}")
        
        # Check fin_quarterly
        fin_path = f"{data_dir}/fin_quarterly.parquet"
        if os.path.exists(fin_path):
            fin = pd.read_parquet(fin_path)
            print(f"\n📌 fin_quarterly.parquet:")
            print(f"   - Total rows: {len(fin):,}")
            print(f"   - Unique customers: {fin['customer_id'].nunique():,}")
            print(f"   - Quarters: {fin['fq_date'].nunique()}")
            print(f"   - Avg EBITDA margin: {(fin['ebitda']/fin['revenue']).mean():.1%}")
            print(f"   - Avg Debt/EBITDA: {(fin['total_debt']/fin['ebitda']).mean():.1f}x")
        
        print("\n✅ All checks passed!")
        
    except Exception as e:
        print(f"\n⚠️  Could not verify data: {e}")
        print("   Run the generate_data.py script first to create the data.")
else:
    print(f"\n⚠️  Data directory not found: {data_dir}")
    print("   Run the following command to generate data:")
    print("   python src/generate_data.py --n-customers 1000 --output-dir data/raw")

print("\n" + "=" * 80)

## 🔧 Step 2: Feature Engineering

Module: `src/feature_engineering.py`

### Overview

Feature Engineering là bước quan trọng nhất trong việc xây dựng mô hình Early Warning System, vì nó chuyển đổi dữ liệu thô từ các bảng tài chính, hành vi tín dụng, và dòng tiền thành các đặc trưng (features) có sức mạnh dự đoán cao. Quá trình này kết hợp kiến thức chuyên môn về tín dụng ngân hàng với kỹ thuật phân tích dữ liệu, tạo ra tập hợp các chỉ số phản ánh đầy đủ tình hình tài chính và rủi ro của khách hàng doanh nghiệp.

Các features được thiết kế dựa trên các nguyên tắc Basel và thực tiễn quản lý rủi ro tín dụng, chia thành 5 nhóm chính: Financial Ratios, Behavioral Features, Cashflow Features, Covenant Breach Flags, và Normalization. Mỗi nhóm phục vụ một mục đích cụ thể trong việc đánh giá khả năng vỡ nợ của khách hàng.

---

### A. Financial Ratios (TTM - Trailing 12 Months)

Các tỷ số tài chính được tính toán dựa trên dữ liệu 12 tháng gần nhất (TTM) để phản ánh xu hướng dài hạn và giảm thiểu ảnh hưởng của biến động ngắn hạn. Chúng ta sử dụng dữ liệu từ 4 quý gần nhất để tính toán các chỉ số tổng hợp này.

#### Liquidity & Coverage Ratios

**Interest Coverage Ratio (ICR)** là chỉ số quan trọng nhất trong đánh giá khả năng trả nợ, được tính bằng EBIT chia cho chi phí lãi vay (Interest Expense). Tỷ số này đo lường khả năng của doanh nghiệp trong việc trả lãi vay từ lợi nhuận hoạt động. Theo thông lệ ngành ngân hàng, ICR dưới 1.5 được coi là mức nguy hiểm, cho thấy doanh nghiệp không đủ khả năng trang trải nghĩa vụ lãi vay từ thu nhập hoạt động.

**Debt Service Coverage Ratio (DSCR)** đo lường khả năng trả cả nợ gốc và lãi từ EBITDA sau khi trừ đi chi phí vốn (CAPEX). Do dữ liệu synthetic không có thông tin chi tiết về khoản trả nợ gốc, chúng ta sử dụng proxy bằng cách ước tính CAPEX là 30% của EBITDA. DSCR dưới 1.2 cho thấy doanh nghiệp gặp khó khăn trong việc đáp ứng các nghĩa vụ nợ.

**Current Ratio** phản ánh thanh khoản ngắn hạn, được tính bằng tài sản ngắn hạn (Current Assets) chia cho nợ ngắn hạn (Current Liabilities). Tỷ số này cho biết khả năng của doanh nghiệp trong việc thanh toán các khoản nợ đến hạn trong vòng 12 tháng tới. Current Ratio dưới 1.0 là dấu hiệu cảnh báo thiếu thanh khoản nghiêm trọng.

#### Leverage Ratio

**Debt-to-EBITDA** đo lường đòn bẩy tài chính, cho biết doanh nghiệp cần bao nhiêu năm EBITDA để trả hết nợ. Tỷ số này được tính bằng tổng nợ (Total Debt) chia cho EBITDA TTM. Theo chuẩn mực ngành, Debt-to-EBITDA vượt quá 4.0 cho thấy doanh nghiệp đang chịu gánh nặng nợ quá mức, làm tăng đáng kể rủi ro vỡ nợ.

#### Working Capital Efficiency

Nhóm chỉ số này đánh giá hiệu quả quản lý vốn lưu động thông qua ba thành phần chính:

**Days Sales Outstanding (DSO)** đo lường số ngày trung bình để thu hồi tiền từ khách hàng, được tính bằng (AR / Revenue) × 365. DSO tăng cao cho thấy doanh nghiệp gặp khó khăn trong việc thu hồi công nợ, có thể dẫn đến thiếu hụt tiền mặt.

**Days Payables Outstanding (DPO)** đo lường số ngày trung bình doanh nghiệp trả tiền cho nhà cung cấp, được tính bằng (AP / COGS) × 365. DPO cao có thể là dấu hiệu tích cực (tận dụng tín dụng thương mại) hoặc tiêu cực (khó khăn thanh khoản).

**Days On Hand (DOH)** đo lường số ngày tồn kho trung bình, được tính bằng (Inventory / COGS) × 365. DOH cao cho thấy hàng tồn kho nhiều, có thể làm gián đoạn dòng tiền.

**Cash Conversion Cycle (CCC)** là chỉ số tổng hợp, được tính bằng DSO + DOH - DPO. CCC đo lường số ngày vốn bị "đóng băng" trong chu kỳ kinh doanh, từ khi trả tiền mua hàng đến khi thu được tiền từ khách hàng. CCC tăng cao cho thấy hiệu quả quản lý vốn lưu động kém, làm dòng tiền xấu đi.

#### Trend Analysis (QoQ)

Ngoài các chỉ số tĩnh, chúng ta còn tính toán xu hướng thay đổi theo quý (Quarter-over-Quarter) cho các chỉ số quan trọng. **delta_dso_qoq** và **delta_ccc_qoq** cho biết sự thay đổi của DSO và CCC so với quý trước, giúp phát hiện sớm các dấu hiệu xấu đi trong quản lý vốn lưu động.

### B. Behavioral Features (Observation Window = 180 ngày)

Các đặc trưng hành vi được trích xuất từ dữ liệu giao dịch hàng ngày trong 180 ngày gần nhất trước ngày đánh giá (as-of date). Những features này phản ánh hành vi sử dụng tín dụng thực tế của khách hàng, thường có sức mạnh dự đoán cao hơn so với các chỉ số tài chính truyền thống vì chúng nắm bắt được các vấn đề thanh khoản và khó khăn tài chính ngay khi chúng phát sinh.

#### Credit Utilization

Tỷ lệ sử dụng hạn mức tín dụng là chỉ số quan trọng phản ánh mức độ phụ thuộc của doanh nghiệp vào nguồn vốn vay ngân hàng. Chúng ta tính toán hai chỉ số chính:

**%util_mean_60d** là tỷ lệ sử dụng hạn mức trung bình trong 60 ngày gần nhất, được tính bằng (Utilized / Limit) trung bình. Chỉ số này cho biết mức độ "căng" về thanh khoản của doanh nghiệp. Utilization rate vượt quá 85% cho thấy doanh nghiệp đang áp sát hạn mức, có nguy cơ thiếu thanh khoản nếu có bất kỳ cú sốc nào.

**%util_p95_60d** là percentile thứ 95 của utilization trong 60 ngày, đo lường đỉnh sử dụng hạn mức. Chỉ số này quan trọng vì nó cho thấy trong những ngày "xấu nhất", doanh nghiệp sử dụng bao nhiêu phần trăm hạn mức, giúp phát hiện các giai đoạn căng thẳng thanh khoản tạm thời.

#### Delinquency Patterns

Days Past Due (DPD) là chỉ số trực tiếp nhất về khó khăn thanh toán. Chúng ta phân tích DPD theo nhiều góc độ:

**dpd_max_180d** là số ngày quá hạn tối đa trong 180 ngày qua. Theo chuẩn mực ngành, DPD vượt quá 30 ngày được coi là early warning signal, trong khi DPD vượt 90 ngày là dấu hiệu rõ ràng của default risk theo định nghĩa Basel.

**dpd_trend_180d** đo lường xu hướng của DPD theo thời gian bằng cách tính slope (hệ số góc) của đường hồi quy tuyến tính giữa DPD và thời gian. Slope dương cho thấy DPD đang có xu hướng tăng dần (tình hình xấu đi), trong khi slope âm cho thấy doanh nghiệp đang cải thiện khả năng thanh toán.

**near_due_freq_7d** đo lường tần suất "gần quá hạn" trong 7 ngày gần nhất, được định nghĩa là tỷ lệ ngày có 0 < DPD < 30. Chỉ số này giúp phát hiện các khách hàng thường xuyên trễ hạn nhưng chưa đến mức quá hạn nghiêm trọng, đây là early warning quan trọng.

#### Credit Limit Breach

**limit_breach_cnt_90d** đếm số lần khách hàng vượt quá hạn mức tín dụng trong 90 ngày gần nhất. Bất kỳ lần vi phạm nào (> 0) đều là dấu hiệu cảnh báo nghiêm trọng, cho thấy doanh nghiệp có nhu cầu vốn vượt quá khả năng được phê duyệt, hoặc hệ thống kiểm soát nội bộ kém.

---

### C. Cashflow Features

Dòng tiền là "huyết mạch" của doanh nghiệp, quan trọng hơn cả lợi nhuận kế toán trong việc dự đoán khả năng vỡ nợ. Chúng ta phân tích dòng tiền vào/ra hàng ngày trong 180 ngày qua để tạo các features:

**inflow_mean_60d** và **outflow_mean_60d** là dòng tiền vào và ra trung bình trong 60 ngày gần nhất. Hai chỉ số này phản ánh quy mô và tính ổn định của hoạt động kinh doanh. Sự chênh lệch lớn giữa inflow và outflow (burn rate cao) là dấu hiệu cảnh báo.

**inflow_drop_60d** đo lường tỷ lệ giảm của dòng tiền vào trong 60 ngày gần nhất so với median 6 tháng, được tính bằng (median_6m - mean_60d) / median_6m. Chỉ số này giúp phát hiện sớm sụt giảm doanh thu, một trong những nguyên nhân chính dẫn đến vỡ nợ. Inflow drop vượt quá 20% cho thấy dòng tiền đang giảm mạnh, cần có hành động can thiệp ngay lập tức.

---

### D. Covenant Breach Flags

Covenant (điều khoản ràng buộc) là các ngưỡng tài chính mà khách hàng phải duy trì theo hợp đồng tín dụng. Vi phạm covenant là early warning signal cực kỳ quan trọng, thường xảy ra trước khi default thực sự diễn ra.

Chúng ta theo dõi ba loại covenant chính: **breach_icr** (vi phạm ICR < 1.5), **breach_dscr** (vi phạm DSCR < 1.2), và **breach_leverage** (vi phạm Debt/EBITDA > 4.0). Mỗi breach flag là biến nhị phân (0/1), cho biết khách hàng có vi phạm covenant tương ứng hay không. Vi phạm bất kỳ covenant nào đều trigger các hành động như renegotiation, tightening điều kiện, hoặc tăng giám sát.

---

### E. Normalization (Sector-Size)

Một trong những thách thức lớn nhất trong credit scoring là so sánh các doanh nghiệp khác nhau về quy mô và ngành nghề. Một SME trong ngành Retail có ICR = 2.0 có thể được coi là tốt, nhưng cùng chỉ số đó với một Large Corporate trong ngành IT lại là mức trung bình hoặc kém.

Để giải quyết vấn đề này, chúng ta áp dụng **Z-score normalization** theo nhóm (Sector, Size_bucket). Mỗi feature được chuẩn hóa bằng cách so sánh với các khách hàng cùng ngành và cùng quy mô:

```
z_score = (value - median_group) / IQR_group
```

Chúng ta sử dụng **Median và IQR (Interquartile Range)** thay vì Mean và Standard Deviation vì chúng robust hơn với outliers, rất phổ biến trong dữ liệu tài chính. Features sau khi normalize có suffix `__zs_sector_size`, ví dụ: `icr_ttm__zs_sector_size`, `dpd_max_180d__zs_sector_size`.

Normalization này mang lại hai lợi ích quan trọng: (1) So sánh công bằng giữa các doanh nghiệp cùng đặc điểm, và (2) Tăng sức mạnh dự đoán của model vì features được điều chỉnh theo context riêng của từng nhóm.

In [ ]:
# Example: Feature Engineering
print("Command to create features:")
print("python src/feature_engineering.py --raw-dir data/raw --asof 2025-06-30 --outdir data/processed")
print("\nKey features created:")
features = [
    "Financial: icr_ttm, dscr_ttm_proxy, debt_to_ebitda, current_ratio, dso, ccc",
    "Behavioral: %util_mean_60d, dpd_max_180d, dpd_trend_180d, limit_breach_cnt_90d",
    "Cashflow: inflow_mean_60d, inflow_drop_60d",
    "Covenant: breach_icr, breach_dscr, breach_leverage",
    "Normalized: *__zs_sector_size versions"
]
for f in features:
    print(f"  - {f}")

## Step 3: Model Training & Calibration

Module: `src/train_baseline.py`

### Overview

Bước này xây dựng mô hình Machine Learning để dự đoán xác suất vỡ nợ (PD) trong 12 tháng tới. Chúng ta sử dụng **LightGBM** làm classifier cơ sở vì khả năng xử lý tốt nhiều features, tự động học được các mối quan hệ phi tuyến, và hỗ trợ class balancing. Sau khi train, mô hình được **calibrate** bằng Isotonic Regression để đảm bảo predicted probabilities phản ánh đúng true probabilities - điều quan trọng cho credit risk management và tuân thủ Basel.

---

### A. LightGBM Configuration

LightGBM được chọn vì xử lý tốt nhiều features có quy mô khác nhau (financial ratios, utilization rates, DPD counts), tự động học feature interactions ("ICR thấp + Utilization cao = Rủi ro cao"), và hỗ trợ class weighting cho imbalanced data (default rate ~5-10%).

**Hyperparameters:**

```python
LGBMClassifier(
    n_estimators=300,           # 300 cây trong ensemble
    learning_rate=0.05,         # Học chậm nhưng ổn định
    num_leaves=15,              # Giới hạn complexity
    max_depth=6,                # Tránh overfitting
    subsample=0.8,              # Row sampling (bagging)
    colsample_bytree=0.8,       # Column sampling
    min_child_samples=10,       # Mỗi leaf ≥ 10 samples
    reg_lambda=0.1,             # L2 regularization
    scale_pos_weight=(1-pos_rate)/pos_rate,  # Auto-balance classes
    objective='binary',
    random_state=42
)
```

**Train-Test Split**: 80% training, 20% holdout test với stratified sampling để đảm bảo default rate đồng đều.

---

### B. Isotonic Calibration (CV=5)

Gradient boosting models thường cho ra **uncalibrated probabilities** - khi model dự đoán PD = 20%, tỷ lệ thực tế default có thể là 15% hoặc 30%. Trong credit risk, điều này nguy hiểm vì các quyết định quan trọng (capital allocation, pricing, provisioning) dựa vào con số PD này.

**Isotonic Regression** là phương pháp calibration non-parametric và monotonic, học hàm mapping từ raw probabilities sang calibrated probabilities với ràng buộc đơn điệu tăng (customer có raw PD cao hơn vẫn có calibrated PD cao hơn). Chúng ta sử dụng 5-fold CV để tránh overfitting:

```python
from sklearn.calibration import CalibratedClassifierCV

calibrated_model = CalibratedClassifierCV(
    base_lgbm,
    method='isotonic',
    cv=5
)
calibrated_model.fit(X_train, y_train)
```

Isotonic Regression được ưu tiên hơn Platt Scaling vì: (1) Không giả định functional form (không cần sigmoid), (2) Đảm bảo ranking không đổi, (3) Performance tốt hơn với sample size lớn (≥ 1000 customers).

---

### C. Risk Tiers & Thresholds

Sau calibration, customers được phân loại vào 3 risk tiers dựa trên **percentile-based thresholds** (thay vì absolute PD cutoffs) để quản lý capacity. Ngân hàng chỉ có đủ nguồn lực để quản lý chặt chẽ một số lượng customers high-risk nhất, nên việc cố định Red tier = top 5% đảm bảo số lượng customers cần intensive monitoring không vượt quá capacity.

**Tier Definitions:**

| Tier | Percentile | Typical PD | Action | Capacity |
|------|-----------|-----------|--------|----------|
| **Red** | Top 5% | ≥ 20% | Họp KH ≤5 ngày; lập cash flow 13 tuần; tighten covenants; watchlist | ~50 KH → 5 RMs |
| **Amber** | Top 5-15% | 5-20% | Soát xét ≤10 ngày; yêu cầu management accounts; hạn chế hạn mức | ~100 KH → 10 RMs |
| **Green** | Bottom 85% | < 5% | Theo dõi định kỳ quarterly; không cần hành động đặc biệt | Portfolio monitoring |

**Threshold Calculation:**

```python
train_probs = calibrated_model.predict_proba(X_train)[:, 1]
red_threshold = np.percentile(train_probs, 95)      # e.g., 0.23
amber_threshold = np.percentile(train_probs, 85)    # e.g., 0.08
```

Thresholds được lưu vào `thresholds.json` và sử dụng nhất quán cho các lần scoring sau.

---

### D. Evaluation Metrics

Model được đánh giá toàn diện qua nhiều metrics, mỗi metric đo lường một khía cạnh khác nhau:

**1. AUC-ROC (Discrimination Power)**  
Đo khả năng phân biệt defaulters vs non-defaulters. AUC = 0.80 nghĩa là 80% trường hợp model sẽ rank đúng (assign PD cao hơn cho defaulter). **Target**: ≥ 0.75 (industry standard).

**2. PR-AUC (Precision-Recall)**  
Quan trọng với imbalanced data (default rate thấp). Precision = % customers được dự đoán default thực sự default. Recall = % defaults thực tế được phát hiện. **Target**: ≥ 0.40 (với base rate ~8%).

**3. KS Statistic (Kolmogorov-Smirnov)**  
Đo maximum separation giữa cumulative distributions của defaulters và non-defaulters. KS = max(TPR - FPR). **Target**: ≥ 0.35 (good discriminatory power).

**4. Brier Score (Calibration Quality)**  
MSE của probabilities: `Brier = (1/N) × Σ(predicted_prob - actual_outcome)²`. Brier nhỏ nghĩa là predictions accurate (nếu dự đoán 10 KH mỗi người PD=20%, lý tưởng có 2 defaults). **Target**: ≤ 0.10. Brier giảm đáng kể sau calibration (từ ~0.12 xuống ~0.08).

**5. Calibration Curve (Reliability Diagram)**  
Visualize calibration: plot mean predicted probability vs actual default rate trong từng bin. Đường lý tưởng là y = x (diagonal).

---

### E. Outputs & Artifacts

**1. Model File**: `model_lgbm.pkl` - Chứa base LightGBM, calibrated model, feature names, và metadata (training date, hyperparameters, test AUC, test Brier).

**2. Scores**: `scores_all.csv` - Predictions cho toàn bộ dataset (train + test) với columns: `customer_id`, `prob_default_12m_base`, `prob_default_12m_calibrated`, `tier`, `is_test`.

**3. Thresholds**: `thresholds.json` - Lưu red/amber/green thresholds và percentiles để dùng cho scoring sau này.

**4. Visualizations**:
- `calibration_lgbm.png`: Reliability diagram (before vs after calibration)
- `pr_curve_lgbm.png`: Precision-Recall curve
- `roc_curve_lgbm.png`: ROC curve  
- `shap_summary.png`: Quick SHAP summary (top 10 features)

Tất cả artifacts được version control để đảm bảo reproducibility và auditability.

In [ ]:
# Example: Train model
print("Command to train LightGBM model:")
print("python src/train_baseline.py --features data/processed/feature_ews.parquet --test-size 0.2 --seed 42 --red-pct 0.05 --amber-pct 0.10 --outdir artifacts/models")
print("\nOutputs:")
print("  - artifacts/models/model_lgbm.pkl (base + calibrated model + features)")
print("  - artifacts/models/scores_all.csv (predictions + tiers)")
print("  - artifacts/models/thresholds.json")
print("  - artifacts/models/calibration_lgbm.png")
print("  - artifacts/models/pr_curve_lgbm.png")
print("  - artifacts/models/shap_summary.csv/png")

## 🔍 Step 4: Model Explainability (SHAP)

Module: `src/explain.py`

### Overview

SHAP (SHapley Additive exPlanations) giải thích contribution của từng feature vào prediction dựa trên game theory. SHAP value dương nghĩa là feature đó tăng xác suất default, âm nghĩa là giảm default risk, và magnitude cho biết mức độ ảnh hưởng. Điều này quan trọng cho Credit Committee (giải thích decisions), RM Team (tư vấn customers cải thiện), và Model Validation (đảm bảo model học đúng patterns).

---

### Global Explainability

**Feature Importance** (`feature_importance.csv`): Mean absolute SHAP values cho mỗi feature, cho biết features nào ảnh hưởng nhất đến model trong toàn bộ portfolio. Ví dụ, `dpd_max_180d__zs_sector_size` thường là feature quan trọng nhất vì DPD là signal mạnh nhất cho default risk.

**SHAP Summary Plot** (`shap_summary.png`): Waterfall plot visualize impact của tất cả features. Mỗi điểm là một customer, màu đỏ = feature value cao, xanh = feature value thấp. Plot này cho thấy không chỉ feature nào quan trọng mà còn direction của impact (high DPD → high risk, high ICR → low risk).

---

### Local Explainability

**Top Drivers per Customer** (`top_drivers_per_customer.csv`): Top 5 features quan trọng nhất cho từng customer cụ thể, giúp trả lời câu hỏi "Tại sao customer C0042 được phân vào Red tier?". Output bao gồm feature name, SHAP value, và actual feature value.

Ví dụ cho customer C0042:
1. `dpd_max_180d__zs_sector_size`: SHAP = +0.52 (value = 120 days) → DPD cao
2. `%util_mean_60d__zs_sector_size`: SHAP = +0.31 (value = 0.95) → Utilization sát hạn mức
3. `icr_ttm__zs_sector_size`: SHAP = +0.20 (value = 0.8) → ICR thấp, khó trả lãi

Với thông tin này, RM có thể tư vấn customer: (1) Clear outstanding payments để giảm DPD, (2) Giảm credit usage hoặc apply for limit increase, (3) Cải thiện profitability hoặc restructure debt.

---

### Dependence Plots

SHAP dependence plots cho key features (`icr_ttm`, `ccc`, `%util_mean_60d`) hiển thị mối quan hệ phi tuyến giữa feature value và SHAP value. Ví dụ, dependence plot của ICR có thể cho thấy: ICR < 1.5 có SHAP values rất cao (risk tăng mạnh), ICR 1.5-3.0 có SHAP giảm dần, ICR > 3.0 có SHAP gần 0 (không còn rủi ro thêm). Những insights này giúp validate model đang học đúng business logic.

---

### Outputs Summary

| File | Type | Purpose |
|------|------|---------|
| `feature_importance.csv` | Global | Ranking features by importance |
| `shap_summary.png` | Global | Visual impact of all features |
| `top_drivers_per_customer.csv` | Local | Top 5 drivers cho từng customer |
| `shap_dependence_*.png` | Global | Phi tuyến relationships |
| `summary.json` | Metadata | Config và stats |

In [ ]:
# Example: Generate SHAP explanations
print("Command to generate SHAP explanations:")
print("python src/explain.py --model artifacts/models/model_lgbm.pkl --features data/processed/feature_ews.parquet --outdir artifacts/shap --max-display 20")
print("\nOutputs:")
print("  - artifacts/shap/feature_importance.csv")
print("  - artifacts/shap/shap_summary.png")
print("  - artifacts/shap/top_drivers_per_customer.csv")
print("  - artifacts/shap/shap_dependence_*.png")
print("  - artifacts/shap/summary.json")

## ⚙️ Step 5-6: [Optional] Re-calibration

### Why Optional?

Bước 3 (train_baseline.py) đã tạo ra một **calibrated model** với percentile-based thresholds (Red = top 5%, Amber = top 5-15%) sẵn sàng cho production. Steps 5-6 chỉ cần thiết khi business muốn **thay đổi threshold strategy** từ percentile-based sang **absolute PD cutoffs** (ví dụ: Red ≥ 20% PD, Amber ≥ 5% PD) để phù hợp với risk appetite hoặc regulatory requirements cụ thể.

Trong thực tế, percentile-based approach thường được ưu tiên vì đảm bảo số lượng customers cần intensive monitoring không vượt quá capacity. Tuy nhiên, một số tổ chức (đặc biệt banks tuân thủ Basel/IFRS 9) yêu cầu absolute thresholds để nhất quán với internal risk rating systems hoặc regulatory reporting.

---

### Step 5: Extract Raw Scores

**Module**: `src/make_scores_raw.py`

Trích xuất raw probabilities từ **base LightGBM** (trước khi áp dụng isotonic calibration trong Step 3) để có baseline scores cho re-calibration process. Output là `scores_raw.csv` chứa uncalibrated predictions cho toàn bộ dataset.

**Why needed?** Re-calibration cần raw scores làm input vì chúng ta sẽ fit một calibrator mới với absolute thresholds khác với calibrator trong Step 3.

---

### Step 6: Re-calibrate with Absolute Thresholds

**Module**: `src/calibrate.py`

Fit lại **Isotonic Regression** trên raw scores với absolute PD cutoffs thay vì percentiles. Process bao gồm: (1) Fit calibrator trên training set, (2) Map raw scores → calibrated PD, (3) Apply absolute thresholds (Red ≥ 20%, Amber ≥ 5%), (4) Save calibrator và thresholds.

**Key difference from Step 3:**
- Step 3: Calibrate → Calculate percentile thresholds → Tiers fixed by % (top 5%, 10%)
- Step 6: Calibrate → Apply absolute PD thresholds → Tiers vary by portfolio quality

**Outputs:**
- `calibrator.pkl`: New isotonic calibrator
- `mapping.csv`: Raw score → Calibrated PD mapping table
- `thresholds.json`: Absolute cutoffs (red: 0.20, amber: 0.05)
- `calibration_full.png`: Reliability diagram
- `pr_curve_full.png`: Precision-Recall curve

**Tradeoff:** Với absolute thresholds, số lượng customers trong Red/Amber tiers có thể biến động theo quality của portfolio (good period → ít Red, bad period → nhiều Red), gây khó khăn cho capacity planning.

## 🎯 Step 7: Production Scoring

Module: `src/scoring.py`

Scoring là bước cuối cùng để đưa model vào production. Script này load trained model, predict PD cho toàn bộ customers dựa trên feature snapshot tại as-of date (ví dụ: 2025-06-30), sau đó phân tier và đưa ra action recommendations. Output được sử dụng trực tiếp bởi RM team và Risk Committee để ra quyết định nghiệp vụ.

---

### Inputs & Outputs

**Inputs:**
1. **Features**: `data/processed/feature_ews.parquet` - Feature snapshot tại as-of date
2. **Model**: `artifacts/models/model_lgbm.pkl` - Trained & calibrated LightGBM
3. **Thresholds**: `artifacts/calibration/thresholds.json` hoặc `artifacts/models/thresholds.json` - Tùy approach (absolute vs percentile)

**Output**: `ews_scored_YYYY-MM-DD.csv` với columns:

| Column | Description | Example |
|--------|-------------|---------|
| `customer_id` | Customer identifier | C0042 |
| `prob_default_12m_calibrated` | PD trong 12 tháng (0-1) | 0.2341 |
| `score_ews` | EWS Score (0-100) | 76.59 |
| `tier` | Risk tier | Red |
| `action` | Recommended action | Họp KH ≤5 ngày; lập cash flow 13 tuần;... |

**EWS Score Formula**: `100 × (1 - PD)` → Score cao = Rủi ro thấp (100 = tốt nhất, 0 = xấu nhất)

---

### Risk Tiers & Actions

| Tier | Criteria | Action | Frequency |
|------|----------|--------|-----------|
| **Green** | PD < 5% (hoặc bottom 85%) | Theo dõi định kỳ; cập nhật BCTC đúng hạn | Quarterly |
| **Amber** | 5% ≤ PD < 20% (hoặc top 5-15%) | Soát xét RM ≤10 ngày; yêu cầu management accounts; kiểm tra công nợ; hạn chế hạn mức | Monthly |
| **Red** | PD ≥ 20% (hoặc top 5%) | Họp KH ≤5 ngày; lập cash flow 13 tuần; xem xét covenant tightening/collateral; watchlist | Weekly |

**Note**: Criteria phụ thuộc vào threshold approach (absolute vs percentile) được chọn ở Steps 3 hoặc 5-6.

---

### Production Workflow

**Monthly Cadence**:
1. **Last day of month**: Chạy scoring script với as-of date = month-end
2. **Day 1-2**: Phân phối report cho RM team và Risk Committee
3. **Day 3-10**: RMs thực hiện actions theo tier (Amber reviews, Red meetings)
4. **Throughout month**: Track action completion và update customer status

**Integration với Banking Systems**:
- **Input**: Features từ core banking system (financial data, credit transactions, cashflow)
- **Output**: EWS scores import vào CRM/Credit Risk systems
- **Alerts**: Auto-trigger emails/notifications cho customers chuyển sang Red tier

**Monitoring**: Track tier migrations month-over-month để identify portfolio trends (improving/deteriorating).

## 📊 Model Performance & Validation

### Expected Performance Metrics (Holdout 20%)

Model được đánh giá trên holdout test set với các metrics sau (computed trong `train_baseline.py` lines 99-104):

| Metric | Target Range | Ý nghĩa | Code |
|--------|-------------|---------|------|
| **AUC-ROC** | 0.75 - 0.85 | Khả năng phân biệt defaulters vs non-defaulters | `roc_auc_score(y_te, p_te)` |
| **PR-AUC** | 0.40 - 0.60 | Performance trên positive class (quan trọng với imbalanced data) | `average_precision_score(y_te, p_te)` |
| **KS Statistic** | 0.35 - 0.50 | Maximum separation giữa cumulative distributions | `ks_score(y_te, p_te)` |
| **Brier Score** | 0.05 - 0.10 | Calibration quality (lower is better) | `brier_score_loss(y_te, p_te)` |

**Calibration Quality**: Reliability curve (predicted probabilities vs actual default rates) nên gần diagonal (y = x). Isotonic calibration cải thiện đáng kể metric này, thường giảm Brier score từ ~0.12 xuống ~0.08. Plots được generate trong `plot_calibration_pr()` function (lines 47-61).

**Precision-Recall Tradeoff**: Red threshold (PD ≥ 20%) có high precision, moderate recall; Amber threshold (PD ≥ 5%) có balanced precision-recall.

---

### Model Monitoring & Maintenance

**Quarterly Reviews** (cần tự implement monitoring scripts):
1. **Performance drift**: Monitor AUC, KS trên new data (target: không giảm > 5%)
   - Re-run `train_baseline.py` trên new data và compare metrics
2. **Population Stability Index (PSI)**: Đo distribution shift của features (target: PSI < 0.15)
   - Formula: `PSI = Σ(actual% - expected%) × ln(actual%/expected%)`
3. **Feature stability**: Check data quality, missing values, outliers
   - Sử dụng data profiling tools hoặc pandas `.describe()`
4. **Recalibration**: Nếu Brier score tăng > 0.10, consider re-fit calibrator
   - Re-run Step 6 (`calibrate.py`) với data mới

**Red Flags Trigger Retraining**:
- AUC drops below 0.70
- Brier score > 0.15
- Large prediction shifts without business explanation (e.g., 10% customers chuyển tier bất thường)
- PSI > 0.25 (severe distribution shift)

## Feature Importance Ranking

Dựa trên SHAP analysis trong `explain.py`, đây là 10 features có impact mạnh nhất đến dự báo default:

| # | Feature Name | Category | Business Interpretation |
|---|--------------|----------|------------------------|
| 1 | `dpd_max_180d__zs_sector_size` | Behavioral | DPD tối đa trong 6 tháng - signal mạnh nhất cho default risk |
| 2 | `%util_mean_60d__zs_sector_size` | Behavioral | Credit utilization trung bình - phản ánh liquidity stress |
| 3 | `icr_ttm__zs_sector_size` | Financial | Interest Coverage Ratio - khả năng trả lãi vay |
| 4 | `debt_to_ebitda__zs_sector_size` | Financial | Financial leverage - mức độ đòn bẩy tài chính |
| 5 | `ccc__zs_sector_size` | Financial | Cash Conversion Cycle - hiệu quả quản lý vốn lưu động |
| 6 | `inflow_drop_60d__zs_sector_size` | Cashflow | Mức giảm doanh thu - suy giảm cashflow |
| 7 | `dpd_trend_180d__zs_sector_size` | Behavioral | Xu hướng DPD tăng - payment behavior đang xấu đi |
| 8 | `breach_icr` | Covenant | Vi phạm covenant ICR - trigger event trực tiếp |
| 9 | `current_ratio__zs_sector_size` | Financial | Current Ratio < 1.0 - nguy cơ thanh khoản ngắn hạn |
| 10 | `delta_ccc_qoq__zs_sector_size` | Financial | Thay đổi CCC theo quý - efficiency đang giảm |

### Phân tích theo Category

- **Behavioral (40%)**: Payment patterns thực tế là predictor mạnh nhất - DPD history và utilization cho signal sớm nhất về khó khăn tài chính
- **Financial (35%)**: Các chỉ số tài chính fundamental (ICR, leverage, liquidity ratios) quan trọng thứ hai
- **Cashflow (15%)**: Revenue trends và cashflow dynamics detect deterioration sớm hơn báo cáo tài chính
- **Covenant (10%)**: Breach events có impact đáng kể nhưng xuất hiện muộn hơn

**Kết luận**: Model ưu tiên behavioral signals vì payment difficulties xuất hiện trước khi financial statements phản ánh đầy đủ. Điều này phù hợp với thực tế risk management trong credit monitoring.

## 🚀 Complete End-to-End Pipeline

### Full Workflow (Development)

```bash
# Step 1: Generate synthetic data
python src/generate_data.py --n-customers 1000 --output-dir data/raw

# Step 2: Feature engineering
python src/feature_engineering.py --raw-dir data/raw --asof 2025-06-30 --outdir data/processed

# Step 3: Train model + calibration
python src/train_baseline.py --features data/processed/feature_ews.parquet --test-size 0.2 --seed 42 --red-pct 0.05 --amber-pct 0.10 --outdir artifacts/models

# Step 4: Generate SHAP explanations
python src/explain.py --model artifacts/models/model_lgbm.pkl --features data/processed/feature_ews.parquet --outdir artifacts/shap --max-display 20

# [Optional] Step 5-6: Re-calibration with absolute thresholds
python src/make_scores_raw.py --features data/processed/feature_ews.parquet --model artifacts/models/model_lgbm.pkl --out data/processed/scores_raw.csv
python src/calibrate.py --input data/processed/scores_raw.csv --red-thr 0.20 --amber-thr 0.05 --outdir artifacts/calibration

# Step 7: Production scoring
python src/scoring.py --features data/processed/feature_ews.parquet --model artifacts/models/model_lgbm.pkl --thresholds artifacts/calibration/thresholds.json --asof 2025-06-30 --outdir artifacts/scoring
```

### Or use Makefile (if configured)

```bash
make requirements    # Install dependencies
make lint           # Check code quality
make format         # Format code with ruff
make test          # Run pytest
```

## 💼 Business Use Cases

### 1. Portfolio Review Meeting (Monthly)

**Input:** `ews_scored_YYYY-MM-DD.csv`

**Analysis:**
```python
import pandas as pd

scores = pd.read_csv('artifacts/scoring/ews_scored_2025-06-30.csv')

# Portfolio distribution
print(scores['tier'].value_counts())
# Green: 850 customers (85%)
# Amber: 100 customers (10%)
# Red:    50 customers (5%)

# High-risk customers requiring immediate action
red_tier = scores[scores['tier'] == 'Red'].sort_values('prob_default_12m_calibrated', ascending=False)
print(f"Red tier: {len(red_tier)} customers with avg PD = {red_tier['prob_default_12m_calibrated'].mean():.1%}")
```

**Actions:**
- **Red tier:** Immediate RM meeting + action plan
- **Amber tier:** Enhanced monitoring + covenant review
- **Green tier:** Standard periodic review

---

### 2. Credit Approval Process

**New loan application từ khách hàng C0523:**

```python
# Get customer's EWS score
customer = scores[scores['customer_id'] == 'C0523'].iloc[0]
print(f"Customer C0523:")
print(f"  - EWS Score: {customer['score_ews']}")
print(f"  - PD 12M: {customer['prob_default_12m_calibrated']:.1%}")
print(f"  - Tier: {customer['tier']}")
print(f"  - Action: {customer['action']}")

# Decision rule
if customer['tier'] == 'Red':
    print("REJECT or require additional collateral")
elif customer['tier'] == 'Amber':
    print("APPROVE with covenant tightening")
else:
    print("APPROVE standard terms")
```

---

### 3. Early Intervention

**Identify deteriorating customers:**

```python
# Compare current month vs last month
current = pd.read_csv('artifacts/scoring/ews_scored_2025-06-30.csv')
previous = pd.read_csv('artifacts/scoring/ews_scored_2025-05-31.csv')

merged = current.merge(previous, on='customer_id', suffixes=('_current', '_previous'))
merged['pd_change'] = merged['prob_default_12m_calibrated_current'] - merged['prob_default_12m_calibrated_previous']

# Customers with PD increasing by > 10pp
deteriorating = merged[merged['pd_change'] > 0.10].sort_values('pd_change', ascending=False)
print(f"Deteriorating customers: {len(deteriorating)}")
```

**Trigger actions:**
- Request updated financials
- Schedule RM meeting
- Review credit limits

---

### 4. SHAP-based Customer Advisory

**Why is customer C0042 in Red tier?**

```python
import pandas as pd

# Load SHAP drivers
drivers = pd.read_csv('artifacts/shap/top_drivers_per_customer.csv')
customer_drivers = drivers[drivers['customer_id'] == 'C0042'].iloc[0]

print("Top 3 risk drivers:")
for i in range(1, 4):
    print(f"{i}. {customer_drivers[f'feat{i}']}: {customer_drivers[f'shap{i}']:.3f} (value={customer_drivers[f'value{i}']})")

# Output:
# 1. dpd_max_180d__zs_sector_size: 0.523 (value=120)
# 2. %util_mean_60d__zs_sector_size: 0.312 (value=0.95)
# 3. icr_ttm__zs_sector_size: 0.201 (value=0.8)
```

**RM Advice to customer:**
1. **DPD 120 days:** Clear outstanding payments immediately
2. **95% utilization:** Reduce credit line usage or apply for limit increase
3. **ICR 0.8:** Improve profitability or restructure debt

## 📈 Artifacts & Outputs Summary

### Directory Structure

```
artifacts/
├── models/
│   ├── model_lgbm.pkl              # Trained model (base + calibrated + features)
│   ├── scores_all.csv              # Training set predictions + tiers
│   ├── thresholds.json             # Percentile-based thresholds
│   ├── calibration_lgbm.png        # Reliability diagram
│   ├── pr_curve_lgbm.png           # Precision-Recall curve
│   └── shap_summary.csv/png        # Quick SHAP summary
│
├── calibration/
│   ├── calibrator.pkl              # Isotonic calibrator (re-fitted)
│   ├── mapping.csv                 # Raw score → Calibrated PD mapping
│   ├── thresholds.json             # Absolute PD thresholds (Red ≥20%, Amber ≥5%)
│   ├── calibration_full.png        # Reliability curve (re-calibrated)
│   └── pr_curve_full.png           # PR curve (re-calibrated)
│
├── shap/
│   ├── feature_importance.csv      # Global feature importance (mean |SHAP|)
│   ├── shap_summary.png            # SHAP waterfall plot
│   ├── top_drivers_per_customer.csv # Local explanations (top 5 features per customer)
│   ├── shap_dependence_*.png       # Dependence plots for key features
│   └── summary.json                # Metadata
│
└── scoring/
    ├── ews_scored_2025-06-30.csv   # Production scores (customer_id, PD, score, tier, action)
    └── thresholds_used.json        # Thresholds applied in this run
```

### Key Files for Different Stakeholders

| Stakeholder | Key Files |
|-------------|-----------|
| **Risk Manager** | `ews_scored_*.csv`, `top_drivers_per_customer.csv` |
| **Credit Committee** | `scores_all.csv`, `shap_summary.png`, `pr_curve_lgbm.png` |
| **Data Scientist** | `model_lgbm.pkl`, `feature_importance.csv`, all plots |
| **Model Validator** | `calibration_*.png`, `thresholds.json`, metrics in console output |
| **Auditor** | All artifacts + `summary.json` for traceability |

## 🔬 Technical Deep Dives

### 1. Why Isotonic Calibration?

**Problem with raw LightGBM probabilities:**
- Overconfident near 0 and 1
- Not well-calibrated for credit risk (regulatory requirement)

**Isotonic Regression:**
- Non-parametric, monotonic calibration
- Preserves ranking (AUC unchanged)
- Improves Brier score and reliability

**Alternative: Platt Scaling (Logistic Regression)**
- Parametric (assumes sigmoid relationship)
- Less flexible than Isotonic
- Use if you need smooth curve

---

### 2. Class Imbalance Handling

**Default rate ~5-10%** → Highly imbalanced

**Strategies applied:**
1. **`scale_pos_weight`** in LightGBM
   - Automatically weights positive class
   - Formula: `(n_negative / n_positive)`
   
2. **Evaluation metrics:** PR-AUC instead of just ROC-AUC
   - ROC-AUC can be misleading with imbalanced data
   
3. **Threshold tuning:** Separate from 0.5
   - Red/Amber thresholds based on business capacity

---

### 3. Feature Normalization (Sector-Size)

**Why normalize by (Sector, Size)?**

```python
# Example: ICR = 2.0 for a SME in Retail
# Is this good or bad?

# Without normalization: Compare to all companies → Looks average
# With sector-size normalization: Compare to SME Retailers → Looks good!

# Implementation:
def sector_size_normalize(df, cols):
    for c in cols:
        grouped = df.groupby(['sector_code', 'size_bucket'])
        median = grouped[c].transform('median')
        iqr = grouped[c].transform(lambda x: x.quantile(0.75) - x.quantile(0.25))
        df[f'{c}__zs_sector_size'] = (df[c] - median) / iqr
    return df
```

**Benefits:**
- Fair comparison (SME vs SME, Corp vs Corp, same sector)
- Robust to outliers (median/IQR instead of mean/std)
- Better predictive power

---

### 4. Label Definition: Event Horizon = 12 Months

**Basel Standard:** PD typically measured over 12-month horizon

**Label rule:**
```python
# Default if: DPD ≥ 90 days for at least 30 consecutive days in next 12M
dpd_90_plus_days = sum(dpd >= 90 for dpd in future_dpd_sequence)
event_h12m = 1 if dpd_90_plus_days >= 30 else 0
```

**Rationale:**
- 90 DPD: Industry standard for "default"
- 30 consecutive days: Avoid transient spikes
- 12M horizon: Align with regulatory reporting

## 🎓 Basel & Regulatory Alignment

### Basel Framework Compliance

**1. PD (Probability of Default) Estimation**
- ✅ 12-month horizon (Basel standard)
- ✅ Through-the-cycle (TTC) calibration via Isotonic Regression
- ✅ Backtesting with holdout set

**2. Key Financial Ratios**
- ✅ **ICR (Interest Coverage Ratio):** EBIT / Interest
- ✅ **DSCR (Debt Service Coverage Ratio):** (EBITDA - CAPEX) / Debt Service
- ✅ **Leverage Ratio:** Total Debt / EBITDA
- ✅ **Liquidity Ratio:** Current Assets / Current Liabilities

**3. Early Warning Indicators**
- ✅ DPD tracking (30, 60, 90+ days)
- ✅ Credit limit breach monitoring
- ✅ Covenant breach flags
- ✅ Cashflow deterioration signals

**4. Model Governance**
- ✅ **Explainability:** SHAP for transparency
- ✅ **Calibration:** Reliability curves
- ✅ **Validation:** AUC, KS, Brier on holdout
- ✅ **Documentation:** All artifacts saved with metadata

---

### Risk Appetite Framework

**Tier Definitions aligned with Risk Appetite:**

| Tier | PD Range | Portfolio Allocation | Risk Appetite |
|------|----------|---------------------|---------------|
| Green | < 5% | 85% | Accept: Standard monitoring |
| Amber | 5-20% | 10% | Tolerate: Enhanced monitoring |
| Red | ≥ 20% | 5% | Mitigate/Exit: Immediate action |

**Capacity Management:**
- Red tier (5%): Max ~50 customers → 5 FTE RM (10 customers/RM)
- Amber tier (10%): Max ~100 customers → 10 FTE RM (10 customers/RM)
- Green tier (85%): Portfolio monitoring only

---

### Regulatory Reporting

**Outputs compatible with:**
- **IFRS 9:** Expected Credit Loss (ECL) calculation
  - PD × LGD × EAD = ECL
  - Model provides PD component
  
- **Basel II/III:** Internal Ratings-Based (IRB) approach
  - PD model for corporate exposures
  - Complement with LGD and EAD models
  
- **Stress Testing:** Scenario-based PD adjustments
  - Re-run model with stressed features
  - Example: Revenue shock, Interest rate shock

## 🛠️ Development & Deployment

### Local Development Setup

```bash
# 1. Clone repository
git clone https://github.com/dylanng3/corporate-credit-ews.git
cd corporate-credit-ews

# 2. Create virtual environment (Python 3.13)
python -m venv .venv
.venv\Scripts\activate  # Windows
source .venv/bin/activate  # Linux/Mac

# 3. Install dependencies
pip install -U pip
pip install -r requirements.txt

# 4. Run linting
make lint

# 5. Format code
make format

# 6. Run tests
make test
```

---

### Testing Strategy

**Unit Tests (`tests/test_data.py`):**
```python
def test_feature_engineering():
    """Test feature calculation logic"""
    # Load sample data
    # Compute features
    # Assert feature values are correct

def test_model_inference():
    """Test model scoring pipeline"""
    # Load trained model
    # Score test cases
    # Assert outputs are valid probabilities [0, 1]
```

**Integration Tests:**
- End-to-end pipeline test
- Data → Features → Model → Scores

---

### Production Deployment Options

**Option 1: Batch Scoring (Recommended)**
```bash
# Cron job: Monthly on last day of month
0 23 L * * python src/scoring.py --features <path> --model <path> --asof $(date +%Y-%m-%d) --outdir artifacts/scoring
```

**Option 2: REST API (Real-time)**
```python
from fastapi import FastAPI
import joblib

app = FastAPI()
model = joblib.load('artifacts/models/model_lgbm.pkl')

@app.post('/predict')
def predict(features: dict):
    X = prepare_features(features)
    prob = model.predict_proba(X)[:, 1][0]
    tier = assign_tier(prob)
    return {'prob': prob, 'tier': tier}
```

**Option 3: Airflow DAG (Orchestration)**
```python
from airflow import DAG
from airflow.operators.bash import BashOperator

dag = DAG('ews_monthly', schedule_interval='0 0 L * *')

generate_features = BashOperator(
    task_id='generate_features',
    bash_command='python src/feature_engineering.py ...'
)

score_customers = BashOperator(
    task_id='score_customers',
    bash_command='python src/scoring.py ...'
)

generate_features >> score_customers
```

---

### Model Versioning

**Recommended: MLflow or DVC**
```bash
# Track model with MLflow
mlflow.log_model(model, 'lgbm_ews_v1')
mlflow.log_metrics({'auc': 0.82, 'ks': 0.45})
mlflow.log_artifacts('artifacts/models')

# Or version with DVC
dvc add artifacts/models/model_lgbm.pkl
git add artifacts/models/model_lgbm.pkl.dvc
git commit -m "Model v1.0 - AUC 0.82"
```

## 📚 Future Enhancements

### 1. Model Improvements

**Advanced Models:**
- ✨ **XGBoost:** Alternative to LightGBM (may perform better)
- ✨ **Neural Networks:** TabNet, FT-Transformer for tabular data
- ✨ **Ensemble:** Stack LightGBM + XGBoost + Logistic Regression

**Feature Engineering:**
- ✨ **Macro variables:** GDP growth, interest rate, sector indices
- ✨ **Time-series features:** ARIMA residuals, trend slopes
- ✨ **Text features:** NLP on management discussions, news sentiment
- ✨ **Network features:** Supply chain relationships, customer concentration

---

### 2. Data Sources

**External Data Integration:**
- 📊 **Credit Bureau:** Payment history from other banks
- 📊 **Market Data:** Stock price (if listed), bond spreads
- 📊 **Alternative Data:** Social media sentiment, web traffic
- 📊 **Geospatial:** Location risk (flood zones, economic zones)

---

### 3. Interpretability Enhancements

**Beyond SHAP:**
- 📖 **LIME:** Local surrogate models
- 📖 **Counterfactuals:** "If ICR increased by 0.5, tier would change from Red to Amber"
- 📖 **Rule extraction:** Decision trees from LightGBM for simple rules
- 📖 **Narrative generation:** Auto-generate credit memos

---

### 4. Operational Features

**Dashboard (Streamlit/Dash):**
```python
import streamlit as st
import pandas as pd

st.title("Corporate Credit EWS Dashboard")

scores = pd.read_csv('artifacts/scoring/ews_scored_latest.csv')

# Tier distribution pie chart
st.plotly_chart(px.pie(scores, names='tier'))

# Customer search
customer_id = st.text_input("Customer ID")
if customer_id:
    customer = scores[scores['customer_id'] == customer_id]
    st.metric("EWS Score", customer['score_ews'].iloc[0])
    st.metric("PD 12M", f"{customer['prob_default_12m_calibrated'].iloc[0]:.1%}")
```

**Alerting System:**
- Email/Slack alerts when customer moves to Red tier
- Weekly summary of tier migrations
- Covenant breach notifications

---

### 5. Model Monitoring

**Drift Detection:**
- **Population Stability Index (PSI):** Track feature distributions
- **Model Performance Tracking:** AUC, KS on rolling windows
- **Prediction Stability:** Variance of predictions month-over-month

**Auto-retraining:**
- Trigger retraining if PSI > 0.15 or AUC drops > 5%
- A/B test new model vs production model
- Gradual rollout with champion-challenger strategy

## 📖 References & Resources

### Academic & Industry Papers

1. **Basel Committee on Banking Supervision**
   - [Basel II: International Convergence of Capital Measurement](https://www.bis.org/publ/bcbs128.htm)
   - PD, LGD, EAD estimation frameworks
   
2. **IFRS 9 - Expected Credit Loss**
   - 12-month vs Lifetime PD
   - Staging models (Stage 1, 2, 3)

3. **Altman Z-Score (1968)**
   - Classic credit scoring model for manufacturing firms
   - Foundation for many modern models

4. **SHAP: Lundberg & Lee (2017)**
   - [A Unified Approach to Interpreting Model Predictions](https://arxiv.org/abs/1705.07874)
   - Game-theoretic feature attribution

---

### Tools & Libraries

**Python Packages:**
- `lightgbm`: Gradient boosting framework
- `shap`: Model explainability
- `scikit-learn`: ML utilities, calibration
- `pandas`, `numpy`: Data manipulation
- `matplotlib`, `seaborn`, `plotly`: Visualization

**Development:**
- `ruff`: Fast Python linter & formatter
- `pytest`: Testing framework
- `cookiecutter-data-science`: Project template

---

### Credit Risk Resources

**Books:**
- *Credit Risk Modeling* by David Lando
- *The Credit Scoring Toolkit* by Raymond Anderson
- *Machine Learning for Asset Managers* by Marcos López de Prado

**Online Courses:**
- Coursera: *Credit Risk Modeling in Python*
- Udemy: *Credit Risk Analytics with Python*

**Websites:**
- [Risk.net](https://www.risk.net) - Industry news
- [Kaggle Credit Risk Datasets](https://www.kaggle.com/search?q=credit+risk)

---

### Contact & Support

**Project Maintainer:** Duong N.C.K  
**Repository:** [github.com/dylanng3/corporate-credit-ews](https://github.com/dylanng3/corporate-credit-ews)  
**License:** MIT License

**For questions or contributions:**
- Open an issue on GitHub
- Submit a pull request
- Email: [contact info if available]

## 🎯 Summary & Conclusion

### What We Built

A **production-ready Early Warning System** for corporate credit risk with:

✅ **End-to-end ML pipeline** (data → features → model → scoring → explainability)  
✅ **Basel-aligned methodology** (12M PD, ICR, DSCR, leverage ratios)  
✅ **LightGBM + Isotonic Calibration** (AUC ~0.75-0.85, well-calibrated probabilities)  
✅ **SHAP explainability** (global + local feature importance)  
✅ **3-tier risk classification** (Green/Amber/Red with actionable recommendations)  
✅ **Production scoring pipeline** (batch scoring with thresholds)  
✅ **Comprehensive artifacts** (models, scores, plots, metadata)

---

### Key Strengths

1. **Comprehensive Feature Engineering**
   - Financial ratios (ICR, DSCR, Leverage, CCC)
   - Behavioral signals (DPD, utilization, trends)
   - Cashflow monitoring
   - Covenant breach tracking
   - Sector-size normalization

2. **Model Quality**
   - Handles class imbalance with `scale_pos_weight`
   - Isotonic calibration for reliable probabilities
   - SHAP for transparency and trust

3. **Business Integration**
   - Clear tier definitions aligned with risk appetite
   - Actionable recommendations for RM team
   - Monthly scoring cadence
   - Customer-level explanations

4. **Code Quality**
   - Modular design (separate scripts for each stage)
   - Type hints and documentation
   - CLI interfaces for all scripts
   - Cookiecutter-data-science template

---

### Success Criteria

**Model Performance:**
- ✅ AUC-ROC ≥ 0.75
- ✅ PR-AUC ≥ 0.40
- ✅ KS ≥ 0.35
- ✅ Brier Score ≤ 0.10

**Business Impact:**
- ✅ Early identification of high-risk customers
- ✅ Reduction in unexpected defaults
- ✅ Efficient RM resource allocation
- ✅ Regulatory compliance (Basel, IFRS 9)

---

### Next Steps

1. **Validation:** Run on real historical data
2. **Backtesting:** Validate predictions against actual defaults
3. **Integration:** Connect to core banking system for live data feeds
4. **Monitoring:** Set up drift detection and performance tracking
5. **Iteration:** Refine features and model based on feedback

---

**This notebook serves as the complete documentation for the Corporate Credit EWS project.**  
**For hands-on experimentation, run the pipeline commands in your terminal or create interactive cells below.**

🚀 **Happy modeling!**